<a href="https://colab.research.google.com/github/vidyabandgar97/A2A/blob/main/detectron2_500_28_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/detectron2_500/

/content/drive/MyDrive/detectron2_500


In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 23 kB/s 
     |████████████████████████████████| 6.6 MB 32.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.5.0+cu101 which is incompatible.
     |████████████████████████████████| 274 kB 4.1 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=d84aadc0db823c88ed150d6e8c7331

In [4]:
!pip install torch==1.9.0 torchvision==0.4.1

     |████████████████████████████████| 831.4 MB 2.8 kB/s 
     |████████████████████████████████| 10.1 MB 22.0 MB/s 
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==1.9.0 and torchvision==0.4.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==1.9.0
    torchvision 0.4.1 depends on torch==1.3.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


In [5]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2 MB 708 kB/s 
     |████████████████████████████████| 49 kB 2.3 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210825-py3-none-any.whl size=60661 sha256=a86ab31828339cf0cbee74ddfcbaf896de7599cb6605c2b53d889c2702a528c5
  Stored in directory: /root/.cache/pip/wheels/53/c4/f8/c4cb07f135845218b019b4a55d8a0470a0f21ee13f8dcd16be
Successfully built fvcore


In [6]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer

In [7]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train2", {}, "/content/drive/MyDrive/detectron2_500/train.json", "/content/drive/MyDrive/detectron2_500/Dataset")
#register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/detectron_train/test/test.json", "/content/drive/MyDrive/detectron_train/test")

In [8]:
from detectron2.data.datasets import register_coco_instances
#register_coco_instances("my_dataset_train2", {}, "/content/drive/MyDrive/detectron2_500/train.json", "/content/drive/MyDrive/detectron2_500/Dataset")
register_coco_instances("my_dataset_val2", {}, "/content/drive/MyDrive/detectron2_500/test.json", "/content/drive/MyDrive/detectron2_500/test_data")

In [9]:
register_coco_instances("my_dataset_test2", {}, "/content/drive/MyDrive/detectron2_500/test.json", "/content/drive/MyDrive/detectron2_500/test_data")

In [10]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
import os

#!mkdir /content/drive/MyDrive/18_Aug_Dataset
#!cp -r /content/Dataset /content/drive/MyDrive/18_Aug_Dataset
#!cp /content/main.json /content/drive/MyDrive/18_Aug_Dataset

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/detectron2_500/output_detect"
cfg.SOLVER.CHECKPOINT_PERIOD = 100
cfg.merge_from_file(model_zoo.get_config_file("Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train2",)
#cfg.DATASETS.TEST = ("my_dataset_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 30000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()
#trainer.test()


[08/28 10:24:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

KeyboardInterrupt: ignored

In [11]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[08/28 11:58:35 d2.data.datasets.coco]: Loaded 190 images in COCO format from /content/drive/MyDrive/detectron2_500/test.json
[08/28 11:58:35 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |  category  | #instances   |   category    | #instances   |
|:----------:|:-------------|:----------:|:-------------|:-------------:|:-------------|
| aeroplane  | 100          |    car     | 99           |     chair     | 99           |
|    cow     | 100          |   person   | 100          | traffic_light | 99           |
|            |              |            |              |               |              |
|   total    | 597          |            |              |               |              |
[08/28 11:58:35 d2.data.common]: Serializing 190 elements to byte tensors and concatenating them all ...
[08/28 11:58:35 d2.data.common]: Serialized dataset takes 0.34 MiB
[08/28 11:58:35 d2.evaluation.evaluator]: Start inference on 190 images
[08/28 11:58:4

OrderedDict([('bbox',
              {'AP': 31.322563034474104,
               'AP-aeroplane': 52.88110034641622,
               'AP-car': 18.31352997428994,
               'AP-chair': 24.517111664069006,
               'AP-cow': 50.9855638148788,
               'AP-person': 15.525501842995611,
               'AP-traffic_light': 25.712570564195076,
               'AP50': 48.54190076993194,
               'AP75': 32.87413676310519,
               'APl': 49.81731369995369,
               'APm': 41.20569763396289,
               'APs': 16.368476936210875}),
             ('segm',
              {'AP': 25.459413280292747,
               'AP-aeroplane': 39.2289481616008,
               'AP-car': 16.56450921271245,
               'AP-chair': 20.012452728273427,
               'AP-cow': 40.64108702985613,
               'AP-person': 13.381601239457169,
               'AP-traffic_light': 22.927881309856524,
               'AP50': 45.654217238147524,
               'AP75': 27.024442760276408,
    

In [12]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_train2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_train2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[08/28 12:00:35 d2.data.datasets.coco]: Loaded 689 images in COCO format from /content/drive/MyDrive/detectron2_500/train.json
[08/28 12:00:36 d2.data.common]: Serializing 689 elements to byte tensors and concatenating them all ...
[08/28 12:00:36 d2.data.common]: Serialized dataset takes 1.35 MiB
[08/28 12:00:36 d2.evaluation.evaluator]: Start inference on 689 images
[08/28 12:00:40 d2.evaluation.evaluator]: Inference done 11/689. 0.4032 s / img. ETA=0:04:36
[08/28 12:00:46 d2.evaluation.evaluator]: Inference done 25/689. 0.3814 s / img. ETA=0:04:16
[08/28 12:00:51 d2.evaluation.evaluator]: Inference done 38/689. 0.3809 s / img. ETA=0:04:12
[08/28 12:00:56 d2.evaluation.evaluator]: Inference done 51/689. 0.3815 s / img. ETA=0:04:07
[08/28 12:01:01 d2.evaluation.evaluator]: Inference done 64/689. 0.3844 s / img. ETA=0:04:05
[08/28 12:01:06 d2.evaluation.evaluator]: Inference done 76/689. 0.3870 s / img. ETA=0:04:03
[08/28 12:01:11 d2.evaluation.evaluator]: Inference done 89/689. 0.3873

OrderedDict([('bbox',
              {'AP': 80.1224579900413,
               'AP-aeroplane': 91.66925261836421,
               'AP-car': 76.20802777754848,
               'AP-chair': 80.81000646123117,
               'AP-cow': 82.87489604964135,
               'AP-person': 81.24655159749277,
               'AP-traffic_light': 67.92601343596976,
               'AP50': 97.3405425710666,
               'AP75': 90.7393695020806,
               'APl': 91.93658270063733,
               'APm': 86.25553514992798,
               'APs': 73.5811477890444}),
             ('segm',
              {'AP': 61.77603837290781,
               'AP-aeroplane': 65.48956853579926,
               'AP-car': 63.24125927298214,
               'AP-chair': 58.38637276055864,
               'AP-cow': 61.875316189144236,
               'AP-person': 60.82387719265814,
               'AP-traffic_light': 60.83983628630445,
               'AP50': 94.72805196559399,
               'AP75': 73.02207236004146,
               '